# Tutorial (2): Tapering-off qubits

Here we show some examples of how to manipulate the tapering-off algorithm using N$_2$/STO-6G.

[(1) Prepare and run UCCSD](#(1)-Prepare-and-run-UCCSD)  
[(2) Speed-up by tapering-off qubits](#(2)-Speed-up-by-tapering-off-qubits)  
[(3) Tapering-off-related methods](#(3)-Tapering-off-related-methods)  



In [1]:
# import necessary modules
from quket import *
from quket.utils import *
import quket.config as cf

mpi4py is not imported. no MPI.


## (1) Prepare and run UCCSD
Run SAUCCSD-VQE to see how log it takes without tapering-off

In [2]:
### Create QuketData for N2 ###
Q = create(basis="sto-6g", 
                 ansatz="sauccsd", 
                 n_orbitals =6, 
                 n_electrons=6, 
                 geometry = "N; N 1 1.098", 
                 mapping = 'bk'
                )
# Run UCCSD
Q.run()

Basis set = sto-6g

*** Geometry ******************************
  N     0.0000000    0.0000000    0.0000000
  N     1.0980000    0.0000000    0.0000000
*******************************************

Symmetry Dooh : D2h(Abelian)
E[FCI]    = -108.669172966971     (Spin = 1   Ms = 0)
E[HF]     = -108.541914960860     (Spin = 1   Ms = 0)


Overwritten attributes  contract_2e  of <class 'pyscf.fci.direct_spin1_symm.FCISolver'>


NBasis = 10
Entered VQE driver
Performing VQE for sauccsd
Number of VQE parameters: 54
Initial configuration: |000000111111>
Convergence criteria: ftol = 1E-09, gtol = 1E-05
Derivatives: Analytical
Circuit order: Exp[T1] Exp[T2] |0>
Initial: E[sauccsd] = -108.541914960860  <S**2> = +0.000000  rho = 1  
      1: E[sauccsd] = -108.646352933729  <S**2> = +0.000001  Grad = 6.26e-01  CPU Time =    0.68297  (0.01 / step)
      2: E[sauccsd] = -108.659301110535  <S**2> = +0.000002  Grad = 3.20e-01  CPU Time =    0.24444  (0.01 / step)
      3: E[sauccsd] = -108.666470234985  <S**2> = +0.000003  Grad = 1.46e-01  CPU Time =    0.23966  (0.01 / step)
      4: E[sauccsd] = -108.668164908414  <S**2> = +0.000004  Grad = 7.34e-02  CPU Time =    0.24352  (0.01 / step)
      5: E[sauccsd] = -108.668556521107  <S**2> = +0.000007  Grad = 7.11e-02  CPU Time =    0.24192  (0.01 / step)
      6: E[sauccsd] = -108.668735126515  <S**2> = +0.000009  Grad = 1.32e-02  CPU Time =    0.24346  (0.01 / step)
      

## (2) Speed-up by tapering-off qubits
Tapering-off algorithm is invoked by specifying the option `taper_off = True` in the input file or `create()`  
or 
by using the method `taper_off()`.

In [3]:
Q.taper_off()

Tapering-Off Results:
List of Tapered-off Qubits:  [0, 1, 3, 5, 7]
Qubit: 0    Tau: 1.0 [Z0 Z2 Z4 Z6 Z8 Z10]
Qubit: 1    Tau: 1.0 [Z1 Z9]
Qubit: 3    Tau: 1.0 [Z3 Z9 Z11]
Qubit: 5    Tau: 1.0 [Z5 Z9 Z11]
Qubit: 7    Tau: 1.0 [Z7 Z11]

States     transformed.
Operators  transformed.
pauli_list transformed.
theta_list transformed.


#### Qubit number is reduced by 5

In [4]:
print("Number of qubits (current) : ", Q.n_qubits)
print("Number of qubits (previous): ", Q._n_qubits)

print_state(Q.init_state, name="\nInitial state with tapered-off mapping")

Number of qubits (current) :  7
Number of qubits (previous):  12

Initial state with tapered-off mapping
    Basis            Coef
| 0000011 > : +1.0000 +0.0000i



In [5]:
# Reset theta_list
Q.theta_list *= 0
# Run
Q.run()

Entered VQE driver
Performing VQE for sauccsd
Number of VQE parameters: 15
Initial configuration: |0111111>
Convergence criteria: ftol = 1E-09, gtol = 1E-05
Derivatives: Analytical
Circuit order: Exp[T1] Exp[T2] |0>
Initial: E[sauccsd] = -108.541914960859  <S**2> = -0.000000  rho = 1  
      1: E[sauccsd] = -108.646352933729  <S**2> = +0.000001  Grad = 6.26e-01  CPU Time =    0.16704  (0.00 / step)
      2: E[sauccsd] = -108.659301110535  <S**2> = +0.000002  Grad = 3.20e-01  CPU Time =    0.08868  (0.00 / step)
      3: E[sauccsd] = -108.666470234984  <S**2> = +0.000003  Grad = 1.46e-01  CPU Time =    0.05748  (0.00 / step)
      4: E[sauccsd] = -108.668164908414  <S**2> = +0.000004  Grad = 7.34e-02  CPU Time =    0.05755  (0.00 / step)
      5: E[sauccsd] = -108.668556521106  <S**2> = +0.000007  Grad = 7.11e-02  CPU Time =    0.05797  (0.00 / step)
      6: E[sauccsd] = -108.668735126515  <S**2> = +0.000009  Grad = 1.32e-02  CPU Time =    0.05709  (0.00 / step)
      7: E[sauccsd] = -

## (3) Tapering-off-related methods
Let's see what methods are available.

#### `taper_off()` method performs a sequence of methods (steps 1 and 2 below) in a black-box manner.
To see what it does step-by-step, let's re-create `QuketData`.

In [6]:
### Create QuketData for N2 ###
Q = create(basis="sto-6g", 
                 ansatz="sauccsd", 
                 n_orbitals =6, 
                 n_electrons=6, 
                 geometry = "N; N 1 1.098", 
                 mapping = 'bk'
                )
#`fci2qubit()` must be run before tapering-off qubits
Q.fci2qubit()

Basis set = sto-6g

*** Geometry ******************************
  N     0.0000000    0.0000000    0.0000000
  N     1.0980000    0.0000000    0.0000000
*******************************************

Symmetry Dooh : D2h(Abelian)
E[FCI]    = -108.669172966971     (Spin = 1   Ms = 0)
E[HF]     = -108.541914960860     (Spin = 1   Ms = 0)
NBasis = 10
FCI in Qubits
(FCI state : E = -108.66917296724462)
      Basis              Coef
| 000000010101 > : +0.9614 +0.0000i
| 000001010001 > : -0.1383 +0.0000i
| 000100010100 > : -0.1383 +0.0000i



### (3.1) Step 1 `tapering.run()`
This detects the symmetries of qubit Hamiltonian,  
and returns various information.

In [7]:
Q.tapering.run(mapping="bk")

Tapering-Off Results:
List of Tapered-off Qubits:  [0, 1, 3, 5, 7]
Qubit: 0    Tau: 1.0 [Z0 Z2 Z4 Z6 Z8 Z10]
Qubit: 1    Tau: 1.0 [Z1 Z9]
Qubit: 3    Tau: 1.0 [Z3 Z9 Z11]
Qubit: 5    Tau: 1.0 [Z5 Z9 Z11]
Qubit: 7    Tau: 1.0 [Z7 Z11]



#### Detected symmetries (`QubitOperator` class)

In [8]:
print(Q.tapering.commutative_taus)

[1.0 [Z0 Z2 Z4 Z6 Z8 Z10], 1.0 [Z1 Z9], 1.0 [Z3 Z9 Z11], 1.0 [Z5 Z9 Z11], 1.0 [Z7 Z11]]


#### Qubits to be removed

In [9]:
print(Q.tapering.redundant_bits)

[0, 1, 3, 5, 7]


#### Unitary (Clifford) operators that transform the Hamiltonian so that it contains only I or X for the redundant_bits above.

In [10]:
print(Q.tapering.clifford_operators)

[0.7071067811865475 [X0] +
0.7071067811865475 [Z0 Z2 Z4 Z6 Z8 Z10], 0.7071067811865475 [X1] +
0.7071067811865475 [Z1 Z9], 0.7071067811865475 [X3] +
0.7071067811865475 [Z3 Z9 Z11], 0.7071067811865475 [X5] +
0.7071067811865475 [Z5 Z9 Z11], 0.7071067811865475 [X7] +
0.7071067811865475 [Z7 Z11]]


#### The eigenvalues of X of redundant_bits are set to those of initial determinant (`det`)

In [11]:
print("Initial determinant (Jordan-Wigner representation) = ", format(Q.det, f"0{Q.n_qubits}b"))
print("Eigenvalues to be used:")
for qubit, eigval in zip(Q.tapering.redundant_bits,  Q.tapering.X_eigvals):
    print("qubit", qubit, " is replaced by", eigval)

Initial determinant (Jordan-Wigner representation) =  000000111111
Eigenvalues to be used:
qubit 0  is replaced by -1
qubit 1  is replaced by 1
qubit 3  is replaced by 1
qubit 5  is replaced by 1
qubit 7  is replaced by 1


### (3.2) Step 2 `transform_***()`
Using the above results, transform each quantity to the reduced-mapping.

`transform_states()` : Transform states in `QuketData` (state, init_state, fci_states, etc.)  
`transform_operators()` : Transform qubit operators in `QuketData` (Hamiltonian, S^2, Number, etc.)  
`transform_pauli_list()` : Transform `QuketData.pauli_list`  
`transform_theta_list()` : Transform `QuketData.theta_list`

Each function is controlled by two arguments:

`backtransform`: Whether to perform backtransformation to the original mapping (default `False`)  
`reduce`: Whether to remove qubits from the simulation (default `True`)  This is usually set to `True`: may be changed to `False` for debugging purposes but doing so is not recommended unless necessary because it could cause problems.

#### Initial state and FCI state transformed and backtransformed.

In [12]:
Q.transform_states()
print_state(Q.init_state, "Reduced mapping for initial state")
print_state(Q.fci_states[0]['state'], "Reduced mapping for FCI state")

States     transformed.
Reduced mapping for initial state
    Basis            Coef
| 0000011 > : +1.0000 +0.0000i

Reduced mapping for FCI state
    Basis            Coef
| 0000011 > : +0.9614 +0.0000i
| 0000110 > : -0.1383 +0.0000i
| 0001011 > : +0.1383 +0.0000i



In [13]:
Q.transform_states(backtransform=True)
print_state(Q.init_state, "Original mapping for initial state")
print_state(Q.fci_states[0]['state'], "Original mapping for FCI state")

States     backtransformed.
Original mapping for initial state
      Basis              Coef
| 000000010101 > : +1.0000 +0.0000i

Original mapping for FCI state
      Basis              Coef
| 000000010101 > : +0.9614 +0.0000i
| 000001010001 > : -0.1383 +0.0000i
| 000100010100 > : -0.1383 +0.0000i



#### Transform operators, pauli_list, and theta_list

In [14]:
Q.transform_operators()
Q.transform_pauli_list()
Q.transform_theta_list()

Operators  transformed.
pauli_list transformed.
theta_list transformed.


#### If there is inconsistency in terms of the quantities being transformed or not transformed, the simulation will fail because of the inconsistent qubit numbers.
When this occurs, Quket is terminated to avoid any confusion.

In [15]:
Q.get_E(Q.fci_states[0]['state'])

Warning!
 Operators are tapered-off [True]
   States are tapered-off [False]
The result below may be nonsense.

 Mismatch of n_qubits between ops (7) and state (12) 



Exception: Error termination of quket.

The above error is intended.


#### One may track which quantity is transformed with `tapered` attribute.

In [ ]:
Q.tapered

In [ ]:
### Since the states are not transformed, re-transform them.
Q.transform_states()

### Now the energy should be safely computed.
Q.get_E(Q.fci_states[0]['state'])